# Project 1 
 
 ## Write a Data Science Blog Post
 
 This project is in completion of the Udacity Data science NanoDegree lesson 1. 

Data used for this project are downloaded from https://www.kaggle.com/airbnb/ for both Boston in Massachusetts, and Seattle in Washington State. According to wikipedia "Seattle is the largest city in both the state of Washington and the Pacific Northwest region of North America" and boston is the "most populous city of the Commonwealth of Massachusetts.., and the 21st most populous city in the United States". This project, using the airbnb data for these two cities, answer questions like : 
 >> 1. What is best property type to list?
 >> 2. The best cancellation policy to adopt?
 >> 3. The essential Amenities needed for every listing?
 >> 4. Predict prices of listing from the data. 

In [ ]:
# importing the python modules and libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from collections import defaultdict
from sklearn.model_selection import train_test_split
from project_1_func import fill_property, fill_bathrooms, fill_bedrooms_beds, fill_zip
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

In [ ]:
# load the boston dataset into dataframes 
boston_calendar = pd.read_csv('~/Udacity_data_science_/lesson_1_project/boston /calendar.csv')
boston_listing = pd.read_csv('~/Udacity_data_science_/lesson_1_project/boston /listings.csv')
boston_reviews = pd.read_csv('~/Udacity_data_science_/lesson_1_project/boston /reviews.csv')

In [ ]:
# loading the seattle dataset into dataframes 
seattle_calendar = pd.read_csv('~/Udacity_data_science_/lesson_1_project/seattle /calendar.csv')
seattle_listing = pd.read_csv('~/Udacity_data_science_/lesson_1_project/seattle /listings.csv')
seattle_reviews = pd.read_csv('~/Udacity_data_science_/lesson_1_project/seattle /reviews.csv')

In [ ]:
# some exploratory analysis on the boston dataframes 
boston_calendar.head()

In [ ]:
boston_calendar.shape # 4 columns and 1308890 rows 

In [ ]:
boston_calendar.date.max()

In [ ]:
# property availabilties
boston_calendar.available.value_counts()

In [ ]:
# boston review dataframe exploratory data analysis 
boston_reviews.head()

In [ ]:
boston_reviews.shape # 6 columns and 63275 rows of reviews 

In [ ]:
# the max review date
boston_reviews.date.max()

In [ ]:
# the earliest review date
boston_reviews.date.min()

In [ ]:
# boston listings dataframe anaylysis
boston_listing.head()

In [ ]:
boston_listing.shape # 95 columns and 3585 rows 

In [ ]:
# making a copy of the boston listing dataframe 

boston_listing_copy = boston_listing.copy()

In [ ]:
# subsetting few columns needed 
boston_listing_df = boston_listing[['id', 'city', 'state', 'zipcode', 'market', 'country',
               'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities',
                'price', 'neighbourhood_cleansed', 'cancellation_policy', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights']]

In [ ]:
# subsetting columns for the analysis 
# subsetting the dataframe to only market in the Boston area 
boston_listing_df = boston_listing_df[boston_listing_df.groupby('market')['market'].transform('size') > 1]

In [ ]:
boston_listing_df.shape # 22 columns and 3568 rows 

In [ ]:
# Handling rows/columns with null values 
boston_listing_df.isnull().any(axis=0)

In [ ]:
# finding the columns with Nulls and number of nulls 
null_columns = boston_listing_df.columns[boston_listing_df.isnull().any()]
boston_listing_df[null_columns].isnull().sum()

In [ ]:
# filling missing values on the property type column  
boston_listing_df = fill_property(boston_listing_df, 'property_type', 'neighbourhood_cleansed')

In [ ]:
boston_listing_df = fill_bathrooms(boston_listing_df, 'bathrooms', 'bedrooms')

In [ ]:
# filling na's on bedrooms column
boston_listing_df = fill_bedrooms_beds(boston_listing_df, 'bedrooms', 'beds')

In [ ]:
# filling na's on bed column
boston_listing_df = fill_bedrooms_beds(boston_listing_df, 'beds', 'bedrooms')

In [ ]:
# since Boston is the most stated city value, and the market area is all stated as Boston
# fillna in the city column with Boston 
boston_listing_df['city'].fillna('Boston', inplace = True) 

In [ ]:
# fill the NaN on the zipcode column 
# delete this line, use null_replace function
boston_listing_df['zipcode'].fillna( boston_listing_df[( boston_listing_df['city'] == 'Boston') & 
                                                        ( boston_listing_df['neighbourhood_cleansed'] 
                                                         == 'Roslindale')]
                                     ['zipcode'][0], inplace=True)

In [ ]:
# dictionary of city values to be replaced 
city_dict = {'Boston (Jamaica Plain)':'Jamaica Plain', 'dorchester':'Dorchester', 'Boston':'Boston',
            'Roslindale':'Roslindale', 'Jamaica plain':'Jamaica Plain', 'Jamaica Plain (Boston)':'Jamaica Plain',
            'Boston (Charlestown)' : 'Charlestown', 'South End, Boston':'South End',
            'Mission Hill, Boston':'Mission Hill', 'Jamaica Plain (Boston)': 'Jamaica Plain', 
             'Jamaica Plain, Boston': 'Jamaica Plain', 'Jamaica Plain, MA':'Jamaica Plain'}

In [ ]:
# replacing the city variables with the proper city name
for key, value in city_dict.items():
    boston_listing_df['city'].replace(key, value, inplace=True)


In [ ]:
# dropping the row with the japanese character in the city column 

boston_listing_df = boston_listing_df[boston_listing_df['city'] != '波士顿']

In [ ]:
# converting the price column to numeric datatype 
price_to_str = lambda x: x.strip().replace('$', '').replace(',', '')
boston_listing_df['price'] = boston_listing_df['price'].apply(price_to_str) #apply the lambda function  
boston_listing_df['price'] = boston_listing_df['price'].apply(pd.to_numeric)  # convert to numeric

In [ ]:
boston_listing_df = boston_listing_df[boston_listing_df.groupby('zipcode')['zipcode'].transform('size') > 1]

In [ ]:
boston_listing_df = fill_zip(boston_listing_df, 'zipcode', 'neighbourhood_cleansed')

In [ ]:
# same process for the seattle listing dataframe 
seattle_listing.shape # 92 columns and 3818 rows 

In [ ]:
# subsetting few columns needed 
seattle_listing_df = seattle_listing[['id', 'city', 'state', 'zipcode', 'market', 'country',
               'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities',
                'price', 'neighbourhood_cleansed', 'cancellation_policy', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights']]

In [ ]:
seattle_listing_df.head()

In [ ]:
seattle_listing_df.market.value_counts()

In [ ]:
# value count of the city columns
seattle_listing_df.city.value_counts()

In [ ]:
# dictionary of city values to be replaced 
city_dict = {'Seattle':'Seattle', 'Phinney Ridge Seattle':'Phinney Ridge', 'Ballard, Seattle':'Ballard', 'seattle':'Seattle'}

In [ ]:
# replacing the city with the proper city name
for key, value in city_dict.items():
    seattle_listing_df['city'].replace(key, value, inplace=True)

In [ ]:
# dropping the row with the japanese character in the city column 

seattle_listing_df = seattle_listing_df[seattle_listing_df['city'] != '西雅图']

In [ ]:
seattle_listing_df.city.value_counts()

In [ ]:
seattle_listing_df.property_type.value_counts()

In [ ]:
# Handling rows/columns with null values 
seattle_listing_df.isnull().any(axis=0)

In [ ]:
# finding the columns with Nulls and number of nulls 
null_columns = seattle_listing_df.columns[seattle_listing_df.isnull().any()]
seattle_listing_df[null_columns].isnull().sum()

In [ ]:
# filling missing values on the property type columns  
seattle_listing_df = fill_property(seattle_listing_df, 'property_type', 'neighbourhood_cleansed')

In [ ]:
# filling na's on bedrooms column
seattle_listing_df = fill_bedrooms_beds(seattle_listing_df, 'bedrooms', 'beds')
# filling na's on bed column
seattle_listing_df = fill_bedrooms_beds(seattle_listing_df, 'beds', 'bedrooms')

In [ ]:
seattle_listing_df = fill_bathrooms(seattle_listing_df, 'bathrooms', 'bedrooms')

In [ ]:
# subsetting the seattle_listing_df only to rows where zipcode is NaN
df = seattle_listing_df[seattle_listing_df['zipcode'].isnull()]

In [ ]:
seattle_listing_df = fill_zip(seattle_listing_df, 'zipcode', 'neighbourhood_cleansed')

In [ ]:
price_to_str = lambda x: x.strip().replace('$', '').replace(',', '')
seattle_listing_df['price'] = seattle_listing_df['price'].apply(price_to_str)#apply the lambda function  
seattle_listing_df['price'] = seattle_listing_df['price'].apply(pd.to_numeric)  # convert to numeric


In [ ]:
# filtering out seattle listings with one listing in an entire zipcode
seattle_listing_df = seattle_listing_df[seattle_listing_df.groupby('zipcode')['zipcode'].transform('size') > 1]

In [ ]:
# making a copy of the seattle_listing_df 
seattle_listing_df_copy = seattle_listing_df.copy()

In [ ]:
# amenities to string
amenities_to_string = lambda x: str(set(x.split(','))).strip().replace('{', '').replace('}', '').replace('"', '')

## QUESTION 1: 
What is the best property type and numbers of bedrooms to list on Airbnb in Seattle or Boston area?

In [ ]:
# concat both dataframe 
df = [boston_listing_df, seattle_listing_df]
listings = pd.concat(df)

In [ ]:
listings['extra_people'].dtypes

In [ ]:
# convert the column extra people to float
listings['extra_people'] = listings['extra_people'].apply(price_to_str) #apply the lambda function  
listings['extra_people'] = listings['extra_people'].apply(pd.to_numeric)  # convert to numeric

In [ ]:
listings['amenities'] = listings['amenities'].apply(amenities_to_string)

In [ ]:
# property type distribution in the dataframe 
listings.property_type.value_counts()

In [ ]:
# selecting property type with more than 80 listings 
listings_property = listings[listings.groupby('property_type')['property_type'].transform('size') > 80]

In [ ]:
listings_property.property_type.value_counts()

In [ ]:
# create the plot
base_color = sns.color_palette()[0]
fig = plt.figure(figsize=(10, 6))
fig.add_subplot(1, 1, 1)
sns.countplot(data = listings_property, x = 'property_type', color = base_color)
plt.xlabel('Property Type', fontsize=15)
plt.ylabel('Number of Property', fontsize=15)
plt.title('Property Type Distribution', fontsize=22)
plt.xticks(rotation=15);



In [ ]:
# value counts for the different bedrooms type
listings.bedrooms.value_counts()

In [ ]:
# selecting number of bedrooms with more than 50 listings in the dataframe
bedroom_listings = listings[listings.groupby('bedrooms')['bedrooms'].transform('size') > 50]

In [ ]:
# count plot of numbers of bedrooms count 
fig = plt.figure(figsize=(10, 6))
fig.add_subplot(1, 1, 1)
sns.countplot(data=bedroom_listings, x='bedrooms')
plt.xlabel('Bedrooms', fontsize=15)
plt.ylabel('Bedroom count', fontsize=15)
plt.title('Bedrooms Distribution', fontsize=25)
plt.xticks(rotation=15);

# Answer:  
From the charts above, it is clear that Apartments and 1 bedrooms are the best sellers for Airbnb in the Boston and Seattle area. 
So anyone looking to start an Airbnb business in these areas should consider listing 1 bedroom apartments. 

# Question 2: 
What is the best cancellation policy to adopt? 

In [ ]:
# using the property_types with more than 80 listings
listings_property.cancellation_policy.value_counts()

In [ ]:
cancellation_group = listings_property.cancellation_policy.value_counts()

In [ ]:
# chart of cancellation policies
fig = plt.figure(figsize=(10, 6))
fig.add_subplot(1, 1, 1)
(cancellation_group/bedroom_listings.shape[0]).plot(kind="bar");
plt.title("Cancellation Policies", fontsize=25);
plt.xlabel('Cancellation percentage', fontsize=15)
plt.ylabel('Cancellation Type', fontsize=15)
plt.xticks(rotation=15);

In [ ]:
# cancellation policies grouped by number of bedrooms
listings_property.groupby(['bedrooms', 'cancellation_policy'])['cancellation_policy'].count()

In [ ]:
# cancellation policies grouped by property_types
listings_property.groupby(['property_type', 'cancellation_policy'])['cancellation_policy'].count()

In [ ]:
listings_property.groupby(['room_type', 'cancellation_policy'])['cancellation_policy'].count()

# Answer:
From the chart and the distribution above, it is obvious most listers choose the Strict cancellation policies. 
However, many others prefered either Flexible or Moderate option.

# Question 3:
What are the essential Amenities needed, before listing the property?


In [ ]:
# cast the amenities column as string

Amenities_to_str = lambda x: str(set(x.split(','))).strip().replace('{', '').replace('}', '').replace('"', '').replace('[','').replace(']', '')
listings['amenities'] = listings['amenities'].apply(Amenities_to_str)

In [ ]:
for i in listings['amenities'][3]:
    print(i)

In [ ]:
# function to count the occurence of an amenities in the amenities column

def total_count(df, col1, col2, look_for):
    '''
    INPUT:
    df - the pandas dataframe you want to search
    col1 - the column name you want to look through
    col2 - the column you want to count values from
    look_for - a list of strings you want to search for in each row of df[col]

    OUTPUT:
    new_df - a dataframe of each look_for with the count of how often it shows up
    '''
    new_df = defaultdict(int)
    #loop through list of ed types
    for val in look_for:
        #loop through rows
        for idx in range(df.shape[0]):
            #if the ed type is in the row add 1
            if val in df[col1][idx]:
                new_df[val] += int(df[col2][idx])
    new_df = pd.DataFrame(pd.Series(new_df)).reset_index()
    new_df.columns = [col1, col2]
    new_df.sort_values('count', ascending=False, inplace=True)
    return new_df

In [ ]:
possible_amenities = ['Free Parking on Premises', 'Essentials', 'Shampoo', 'Iron', 'Internet', 'Carbon Monoxide Detector', 'Kitchen', 'TV', 'Gym', 'First Aid Kit', 'Laptop Friendly Workspace', 
    'Wireless Internet', 'Dryer', 'Hair Dryer', 'Air Conditioning','Breakfast','Smoke Detector', 'Safety Card', 'Fire Extinguisher', 'Washer', 'Hangers', 'Heating', 'Indoor Fireplace'
   'Fire Extinguisher', 'Smoke Detector', 'Safety Card','Washer', 'Essentials','Internet', 'Heating', 'Dryer', 'Carbon Monoxide Detector', 'Kitchen','Family/Kid Friendly','Indoor Fireplace','Shampoo']

def clean_and_plot(df, title='Basic Amenities needed', plot=True):
    '''
    INPUT 
        df - a dataframe holding the CousinEducation column
        title - string the title of your plot
        axis - axis object
        plot - bool providing whether or not you want a plot back
        
    OUTPUT
        study_df - a dataframe with the count of how many individuals
        Displays a plot of pretty things related to the CousinEducation column.
    '''
    study = df['amenities'].value_counts().reset_index()
    study.rename(columns={'index': 'Amenities Type', 'amenities': 'count'}, inplace=True)
    study_df = total_count(study, 'Amenities Type', 'count', possible_amenities)

    study_df.set_index('Amenities Type', inplace=True)
    if plot:
        (study_df/study_df.sum()).plot(kind='bar', legend=None);
        plt.title(title);
        plt.show()
    props_study_df = study_df/study_df.sum()
    return props_study_df
    
props_df = clean_and_plot(seattle_listing)

In [ ]:
props_df

# Answer:
Every or most of the listings in these market area, either have Internet, 
Heating and Kitchen. Clearly internet is an essential amenities to have. 

# Question 4:

Predicting price of listings?

In [ ]:
listings.columns

In [ ]:
df = listings[[ 'zipcode', 'market',  'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bed_type', 'neighbourhood_cleansed',
       'cancellation_policy', 'guests_included', 'extra_people', 'price']]

In [ ]:
# checking dtypes of the selected columns 
df.dtypes

In [ ]:
#Pull a list of the column names of the categorical variables
# create dummy variables from categorical columns 

def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
    '''
    for col in  cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, dummy_na=dummy_na)], axis=1)
        except:
            continue
    return df

In [ ]:
#Dropping where the price has missing values
df  = df.dropna(subset=['price'], axis=0)

#Pull a list of the column names of the categorical variables
df_1 = df.select_dtypes(include='object')

cat_cols_lst = df_1.columns

df_new = create_dummy_df(df, cat_cols_lst, dummy_na=False) #Use your newly created function

# Show a header of df_new to check
print(df_new.shape)

In [ ]:
# checking if the dummy columns are greated
df_new.columns

In [ ]:
# select the dependent and independent variables
X = df_new.drop('price', axis=1)
y = df_new['price']

In [ ]:
# splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=1)

In [ ]:
# mutiple linearRegression
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
# apply the model to predict listing price
y_pred = regressor.predict(X_test)


In [ ]:

# model performance using r2 score
r2_score(y_test, y_pred) # perform at 57% accuracy